# 1. 程序入口：int main (int argc, char **argv)
<font size=4 face="微软雅黑"> 
   - 位置：
        - GEBSort.cxx
   - 作用：<font color="Red" >
        - 全局声明TFile、TTree以及要存储的所有变量　</font> 
            - 接收参数（输入文件、输出的root文件、log文件）
            - 初始化参数 <font color="Red" >   
            - 主函数中,调用int GEBacq (char *ChatFileName)函数之前，实现TFile、TTree及其Branch  </font> 
    
       - 调用int GEBacq (char *ChatFileName)函数，该函数的输入参数为GEBSort.chat文件名（文件内存储着事件个数 ，多普勒修正的beta值，各种数据分析程序运行的开关，刻度文件的名字,可以通过＃、；屏蔽，关闭开关）<font color="Red" >   
       - 退出主函数之前，写 tree，关闭文件  </font> 

</font>

# 2. int GEBacq (char *ChatFileName)
<font size=4 face="微软雅黑"> 
   - 位置：
        - GEBSort.cxx
   - 作用：
       - 调用int GEBSort_read_chat (char *name)函数读取GEBSort.chat文件，获取文件中的参数，并按照文件中的内容控制相应的开关。
       - 打开Merge之后的文件
       - 建立root文件
       - 根据开关情况，执行不同的分析程序，目前运行的程序打开的开关：
            - Pars.do_bin_dgs（Ge、BGO分析）
            - Pars.do_bin_XA（X-array(clover)分析）
            - Pars.do_bin_dfma（DSSD、SiBox、PPAC分析）
       - 调用顺序：
            - sup_dgs ();  bin_dgs(GEB_Event * GEB_event); exit_dgs()
            - sup_XA (); int bin_XA (GEB_EVENT * GEB_event); int exit_XA ()
            - sup_dfma ();　int bin_dfma (GEB_EVENT * GEB_event)；int exit_dfma()
    
  - <font color="Red" >在读完事件数据，调用bin_* 函数之前对所有变量进行初始化 </font>
  - <font color="Red" >在读下一个事件之前Fill　TTree </font>
</font>  

## 2.1 bin_dgs.c(Ge、BGO分析)
<font size=4 face="微软雅黑"> 
   - 全局：
       - 声明直方图，建立TFile、TTree
   - sup_dgs()
       - 作用：
           - 建立TTree的Branch
           - 实现直方图
           - 读取刻度文件(Ge_Md.cal)（Ge：１～110）
       - 方法：
           - void getcal (char *file)
               - id = i
               - Pth[i] 
               - BLPth[i]
               - ath[i] (gain)
               - bth[i] (offset)
   - bin_dgs(GEB_Event * GEB_event)
       - 作用：
           - Ge 多普勒修正、刻度、BGO反康分析
       - 方法：
           - 对当前事件GEB_event的GEB_event->mult重事件进行循环
                - 寻找事件类型GEB_event->ptgd[i]->type == GEB_TYPE_DGS(Ge/BGO)的事件
                - 确定Ge/BGO的事件重数ng
                - 将相应事件依次存入DGSEvent数组中
                    - int DGSEvDecompose_v3_ds (unsigned int *ev, int len, DGSEVENT * DGSEvent)
           - 循环DGSEvent数组中存储的ng个事件
                - 挑选出Ge事件（DGSEvent[i].tpe == GE）
                - 探测器代号记为gsid = DGSEvent[i].tid
                - 没有多普勒修正的结果记为DGSEvent[i].ehi_nodop，做过多普勒修正的结果记为DGSEvent[i].ehi，
                - 用DGSEvent[i].flag是否有相应的BGO打火(abs (tdiff) <= 50)（１表示有，０表示没有）
                - 将打火时间与文件开始的时间做差，单位转成秒(DGSEvent[0].event_timestamp - EvTimeStam0) / 100000000)获得计数情况时间谱，将时间按一定单位时间（１ｓ）分bin，获得计数率。
           - Fill相应的直方图
           - <font color="Red" >　Ge阵列数据获取 </font>
   - exit_dgs()
        - 作用
            - 将TTree写入TFile文件并关闭文件 
 </font>

## 2.2 bin_XA.c(分析)
<font size=4 face="微软雅黑"> 
   - 全局：
       - 声明直方图
   - sup_XA()
       - 作用：
           - 实现直方图
           - 读取map文件(XA_map.dat)（没用）
           - 读取刻度文件（xa_ehi.cal）（Clover:1～20）
           - 读取文件（xa_pz.cal）（提取道值）
       - 方法：
           - void XAgetcal (char *file)
               - id = i
               - XAehigain[i] (gain)
               - XAehioffset[i] (offset)
               - XAPZ[i]（xa_pz.cal）
   - int bin_XA (GEB_EVENT * GEB_event)
       - 作用：
           - Clover 刻度
       - 方法：
           - 对当前事件GEB_event的GEB_event->mult重事件进行循环，寻找事件类型GEB_event->ptgd[i]->type ==24，将GEB_TYPE_XA赋值给GEB_event->ptgd[i]->type，即把24换成GEB_TYPE_XA
           - 对当前事件GEB_event的GEB_event->mult重事件进行循环
                - 寻找事件类型GEB_event->ptgd[i]->type == GEB_TYPE_XA(Clover)的事件
                - 确定Clover的事件重数XAng
                - 并将相应事件依次存入XAEvent（extern DGSEVENT XAEvent[MAXCOINEV]）数组中
                    - int DGSEvDecompose_v3 (unsigned int *ev, int len, DGSEVENT * XAEvent, int tlkup[], int tid[])(在jta.c)。
           - 循环XAEvent[]数组中存储的XAng个事件
                - 挑选出Ge事件（XAEvent[i].tpe == GE）
                - 探测器代号记为gsid = XAEvent[i].tid
                - 将打火时间与文件开始时间做差，单位转成秒((XAEvent[0].event_timestamp - EvTimeStam0) / 100000000)获得计数情况时间谱，将时间按一定单位时间（１ｓ）分bin，获得计数率。
                - 能量刻度(545行)
                - 储方式与dgs一致，没有多普勒修正的结果记为XAEvent[i].ehi_nodop，做过多普勒修正的结果记为XAEvent[i].ehi（参数均为0，修正前后结果一致）
                - AEvent[i].flag是否有相应的BGO打火（１表示有，０表示没有），由于没做反康分析，XAEvent[i].flag都是０。
           - Fill相应的直方图
           - 依据多重事件，建立gamma-gamma符合的二维矩阵（有问题）
           - 绘制gamma-gamma符合事件的时间谱
           - 其他分析
   - exit_dgs()
        - 作用
            - 输出一些结束的信号 
 </font>

## 2.3 bin_dfma.c(分析)
<font size=4 face="微软雅黑"> 
   - 全局：
       - 声明直方图
   - sup_XA()
       - 作用：
           - 实现TFile（存储衰变链信息）、TTree及其Branch(存储衰变链的信息)、直方图（存在./ROOT_FILES/run$RUN.root 文件中）
           - 读取DSSD正面刻度数据文件(MAP_FILES/dssd_fr_calib_th.map)(1～160)
               - strip
               - map_fr[strip].phystrip = phystrip
               - map_fr[strip].thr = thr
               - map_fr[strip].off = off
               - map_fr[strip].gain = gain
               - map_fr[strip].baseline = baseline
           - 读取DSSD背面刻度数据文件(MAP_FILES/dssd_ba_calib_th.map)(161～320)
               - strip
               - map_ba[strip].phystrip = phystrip
               - map_ba[strip].thr = thr
               - map_ba[strip].off = off
               - map_ba[strip].gain = gain
               - map_ba[strip].baseline = baseline
           - 读取clover刻度文件（MAP_FILES/clovers.map）（Clover:1～20, 没用）
               - strip
               - clmap[strip].gain = gain
               - clmap[strip].off = off;
           - 读取SiBox的map文件（MAP_FILES/sibox_calib_th.map）（1～54）
               - strip
               - map_box[strip].phystrip = phystrip
               - map_box[strip].thr = thr
               - map_box[strip].off = off
               - map_box[strip].gain = gain
               - map_box[strip].baseline = baseline
           - 获取eftofG.root文件中存储的门
               - eftof = (TCutG*) g->Get("eftofGate");
           - 读取map文件（map_z115.dat）[i2 = fscanf(fp, "\n%i %i %i %s", &i1, &i7, &i8, str)]
               - 通道号：i1(1010～3000)
               - 探测器种类代号：tlkup[i1] = i7
               - 探测器id：tid[i1] = i8;
               - 探测器种类名称：str
                   - 1---Ge
                   - 2---BGO
                   - 3---SIDE
                   - 4---AUX
                   - 5---DSSD
                   - 6---FP
                   - 7---XARRAY
                   - 8---CHICO2
                   - 12---SIBOX（SIB）
           - load_ranges()读取alpha_ranges.txt（没找到？？？）
   - int bin_dfma(GEB_EVENT* GEB_event)
       - 作用：
           - 分析
       - 方法：
           - 对当前事件GEB_event的GEB_event->mult重事件进行循环
               - 将GEB_event->ptgd[i]->type对应的探测器种类名称赋值给strg
               - 寻找事件类型GEB_event->ptgd[i]->type ==16即GEB_TYPE_DFMA事件
               - 将相应事件依次存入DFMAEvent[nsubev]数组中
                   - extern int DFMAEvDecompose_v3(unsigned int* ev, int len, DFMAEVENT* DFMAEvent)
               - 挑选DFMAEvent[nsubev].tpe == DSSD的事件
                   - 记录该种类事件数为ndssd
               - 挑选DFMAEvent[nsubev].tpe == FP
                   - 记录该种类事件数为nfp
               - 记录dfna事件数为ndfma(DSSD,FP),nsubev(GEB_event->ptgd[i]->type == 16(DFMA))
           - 能量道值记为DFMAEvent[i].ehi
           - 时间记为DFMAEvent[i].LEDts
           - 循环挑选出来的DFMA事件
               - 从其中挑选DSSD正面事件（DFMAEvent[i].tpe == DSSD　&& DFMAEvent[i].tid < BATIDLOW）
               - 正面结果进行刻度(加抖动)
                   - cal_e = double(map_fr[DFMAEvent[i].tid].gain) * (DFMAEvent[i].ehi + double(rand()) / RAND_MAX      - 0.5) + double(map_fr[DFMAEvent[i].tid].off)
               - Ma　DSSD正面数据获取 </font>
               - 提取正面信号的最大值记为dssd_fr_emax，事件编号记为dssd_fr_subev = i
           - 再次循环挑选出来的DFMA事件
               - 从其中挑选DSSD背面事件（DFMAEvent[i].tpe == DSSD　&& DFMAEvent[i].tid > FRTIDHIGH）
               - 对背面结果进行刻度(加抖动)
                   - cal_e = double(map_ba[DFMAEvent[i].tid].gain) * (DFMAEvent[i].ehi + double(rand()) / RAND_MAX      - 0.5) + double(map_ba[DFMAEvent[i].tid].off)
               - <font color="Red" >　DSSD背面数据获取 </font>
               - 提取正面信号的最大值记为dssd_ba_emax，事件编号记为dssd_ba_subev = i  
           - 把所有DSSD事件的DFMAEvent[i].ehi换成能量刻度之后的结果
           - 再次循环挑选出来的DFMA事件
               - 挑选SIBOX事件（DFMAEvent[i].tpe==SIBOX）
                   - 刻度能量
                       - cal_e = double(map_box[DFMAEvent[i].tid].gain) * (DFMAEvent[i].ehi + double(rand()) / RANDMAX - 0.5) + double(map_box[DFMAEvent[i].tid].off)
                   - <font color="Red" >　SiBox数据获取 </font>
                   - 记录信息：
                       - SiBox的事件重数
                       - 时间：sib_ts[n_sib] = DFMAEvent[i].LEDts
                       - 刻度后的能量：sib_e[n_sib] = cal_e
                       - SiBox的id:sib_tid[n_sib] = DFMAEvent[i].tid
                       - 最大的SiBox能量：sib_emax = cal_e
               - 挑选FP事件（DFMAEvent[i].tpe==FP）
                   - 挑选MWPC left(DFMAEvent[i].tid==LCH && DFMAEvent[i].ehi > 0)
                       - left_subev = i;(在DFMA中的序号)
                       - cl_pu = DFMAEvent[i].pu;(????)
                       - tdssdmcp_fr = (DFMAEvent[i].LEDts) -  (tdssd_fr);
                       - tdssdmcp_ba = (DFMAEvent[i].LEDts) -  (tdssd_ba);
                       - cl=DFMAEvent[i].ehi;(能量道值)
                       - ts_left = DFMAEvent[i].LEDts;
                       - wheel_left = DFMAEvent[i].wheel;
                   - 挑选MWPC right(DFMAEvent[i].tid==RCH && DFMAEvent[i].ehi > 0)
                       - right_subev = i;(在DFMA中的序号)
                       - cr_pu = DFMAEvent[i].pu;(????)
                       - tdssdmcp_fr_r = (DFMAEvent[i].LEDts) -  (tdssd_fr);
                       - tdssdmcp_ba_r = (DFMAEvent[i].LEDts) -  (tdssd_ba);
                       - cr=DFMAEvent[i].ehi;(能量道值) 
                   - 挑选MWPC up(DFMAEvent[i].tid==UPCH && DFMAEvent[i].ehi > 0)
                       - up_subev = i;(在DFMA中的序号)
                       - cup = DFMAEvent[i].ehi;(能量道值)                       
                       - wheel_up=DFMAEvent[i].wheel;(????)
                   - 挑选MWPC down(DFMAEvent[i].tid==DOWNCH && DFMAEvent[i].ehi > 0)
                       - down_subev = i;(在DFMA中的序号)
                       - cdown = DFMAEvent[i].ehi;(能量道值) 
                   - 在其中选出PPAC事件（DFMAEvent[i].tid == PPACDE）
                       - ppac_de = DFMAEvent[i].ehi;
                       - ppac_t = DFMAEvent[i].LEDts;
                       - ppacde_pu = DFMAEvent[i].pu;
                       - tdssdppacde_fr = (DFMAEvent[i].LEDts) -  (tdssd_fr);
           - 循环XAEvent[i]提取数据
                - <font color="Red" >　clover数据获取 </font>
           - 确定MWPC位置：
                - csum = cl + cr;
                - crat = cl - cr;
                - craty=cup-cdown;
                - csumy=cup+cdown;
   - exit_dgs()
        - 作用
            - 写tree 
 </font>

# 3.数据情况
<font size=4 face="微软雅黑"> 
   - 3.1 Ge 阵列（bin_dgs.c）(Ge/BGO～110)
        - 3.1.1 程序数据：
           - dgs打火个数：ng(868)
           - dgs打火情况：DGSEvent[ng]
           - dgs打火的时间信息：DGSEvent[i].event_timestamp（10 ns）
           - Ge：DGSEvent[j].tpe == Ge
           - Ge打火的能量道值： (1196)
           - Ge打火的能量刻度结果:(1196)
           - Ge探头的角度：d1(1204)
           - Ge探头代号：DGSEvent[j].tid
           - BGO：DGSEvent[j].tpe == BGO
           - BGO探头代号：DGSEvent[j].tid(与Ge探头一一对应，只提取是否响应)
        - 3.1.2 数据提取情况：
           - gehit (Int_t)
           - ge_c[hitge] (Double_t)
           - ge_e[hitge] (Double_t)
           - ge_t[hitge] (ULong64_t)
           - ge_id[hitge] (Int_t)
           - ge_angle[hitge] (Double_t)
           - ge_tBGO[hitge] (ULong64_t)(0标记BGO未打火)
        - 3.1.3 数据问题
           - 存在Ge探测器打火，BGO多次打火的情况（0.3％（500ns的时间窗））
    - 3.2 Xarray(bin_XA.c)(clover)
        - 3.2.1 程序数据
            - XA打火个数：XAng(419)
            - XA打火情况：XAEvent[XAng]（extern DGSEVENT XAEvent[MAXCOINEV]）
            - XA打火的时间信息：XAEvent[0].event_timestamp
            - XAEvent[i].tpe == GE
            - XA打火的能量道值Energy（548）
            - XA打火的能量刻度结果：XAEvent[i].ehi = Energy
            - XA探头代号：XAEvent[i].tid
        - 3.2.2 数据提取情况：
            - cloverhit
            - clover_c[cloverhit]
            - clover_e[cloverhit]
            - clover_t[cloverhit]
            - clover_id[cloverhit]
        - 3.2.3　提取方法
            - 由于bin_XA()函数会在分析过程中多次调用，故clover数据在bin_dfma()函数中通过XAEvent[XAng]读取;
   - 3.3 dssd(bin_dfma.c)
        - 3.3.1 程序数据
            - dfma打火情况：DFMAEvent[nsubev]
            - DFMAEvent[i].tpe == DSSD
                - 时间信息：DFMAEvent[i].LEDts
                - 道数值：DFMAEvent[i].ehi
                - 刻度结果：cal_e 
                - 探头代号：DFMAEvent[i].tid
                - 探头物理代号：map_fr[DFMAEvent[i].tid].phystrip
                - 正面：DFMAEvent[i].tid < BATIDLOW
                - 背面：DFMAEvent[i].tid > FRTIDHIGH
            - DFMAEvent[i].tpe == SIBOX
            - DFMAEvent[i].tpe XAEvent[j].ehi== FP
                - DFMAEvent[i].tid==PPACDE && DFMAEvent[i].ehi > 0
            - Xarray
                - for(j=0; j<XAng; j++)  XAEvent[j].ehi
        - 数据提取情况：
            - dssd
                - dssdfhit
                - dssdf_c[dssdfhit]
                - dssdf_e[dssdfhit]
                - dssdf_t[dssdfhit]
                - dssdf_id[dssdfhit]
                - dssdf_pid[dssdfhit]
                - dssdbhit
                - dssdb_c[dssdbhit]
                - dssdb_e[dssdbhit]
                - dssdb_t[dssdbhit]
                - dssdb_id[dssdbhit]
                - dssdb_pid[dssdbhit]
            - SiBox
                - siboxhit
                - sibox_c[siboxhit]
                - sibox_e[siboxhit]
                - sibox_t[siboxhit]
                - sibox_id[siboxhit]
            - PPAC
                - ppac_de
                - ppac_t

 </font>

# ４.RDT分析程序（dfma）
<font size=4 face="微软雅黑"> 
   - 3.1 DSSD响应(if((dssd_fr_emax != 0) && (dssd_ba_emax != 0)))
        - 3.1.1 Recoil(if(((cl != 0) || (cr != 0) || (ppacde!= 0)) && (dssd_fr_emax > 10000) && (dssd_ba_emax > 10000) && (dssd_fr_emax < 50000) && (dssd_ba_emax < 50000)))
           - 参数
               - cl=DFMAEvent[i].ehi;(DFMAEvent[i].tpe == FP && DFMAEvent[i].tid==LCH && DFMAEvent[i].ehi > 0)(LMWPC)
               - cr=DFMAEvent[i].ehi;(DFMAEvent[i].tpe == FP && DFMAEvent[i].tid==RCH && DFMAEvent[i].ehi > 0)(RMWPC)
               - ppac_de = DFMAEvent[i].ehi;(DFMAEvent[i].tpe == FP && DFMAEvent[i].tid==PPACDE && DFMAEvent[i].ehi > 0)(PPAC/MWPC DE)
               - dssd_fr_emax(DSSD正面能量最大值（KeV）)
               - dssd_fr_subev(DSSD正面最大值在DFMA中的序号)
               - tdssd_fr（DSSD正面最大值响应的时间）
               - dssd_ba_emax(DSSD背面能量最大值（KeV）)
               - dssd_ba_subev(DSSD背面最大值在DFMA中的序号)
               - tdssd_ba（DSSD背面最大值响应的时间）
           - 数据
               - DSSD正面
                   - r_fr_emax = dssd_fr_emax;
                   - s_r_fr = DFMAEvent[dssd_fr_subev].tid;
                   - r_fr_ts = DFMAEvent[dssd_fr_subev].LEDts;
                   - r_fr_subev = dssd_fr_subev = i(dfma);
                   - r_fr_PU = DFMAEvent[dssd_fr_subev].pu;
               - DSSD背面
                   - r_ba_emax = dssd_ba_emax;
                   - s_r_ba = DFMAEvent[dssd_ba_subev].tid - NUMFR;
                   - r_ba_ts = DFMAEvent[dssd_ba_subev].LEDts;
                   - r_ba_subev = dssd_ba_subev;
                   - r_ba_PU = DFMAEvent[dssd_ba_subev].pu;
           - 处理(叠加信号)：
               - if ((s_r_fr!=0)&&(s_r_ba!=0))
                    - if ((r_fr_PU > 1)&&(r_ba_PU > 1))
                        - 参数：
                            - r_fr_PU = DFMAEvent[dssd_fr_subev].pu;
                            - r_ba_PU = DFMAEvent[dssd_ba_subev].pu;
                        - diff = fd_ba_ts - fd_fr_ts;
                    - if ((r_fr_PU > 1)&&(r_ba_PU > 1)&&(diff<5)&&(diff>-5))
                        - rec_PU=1;
                        - fdecEv=1;
                        - s_fd_fr=s_r_fr;
                        - s_fd_ba=s_r_ba;
        - 3.1.2 Decay(if( (cl == 0) && (cr == 0) && (ppacde ==0) && (dssd_fr_emax < 10000)  && (dssd_ba_emax < 10000)))
           - 数据
               - DSSD(正背面数据结构一样)
                   - d_fr_emax = dssd_fr_emax;
                   - s_d_fr = DFMAEvent[dssd_fr_subev].tid;
                   - s_d_fr_phys = map_fr[DFMAEvent[dssd_fr_subev].tid].phystrip;
                   - d_fr_subev = dssd_fr_subev;
                   - d_fr_ts = DFMAEvent[dssd_fr_subev].LEDts;
                   - d_fr_PU = DFMAEvent[dssd_fr_subev].pu;
               - SiBox(n_sib>0)
                   - dssdfrsi_t = double(DFMAEvent[dssd_fr_subev].LEDts) - double(sib_ts[i]);
                   - dssdbasi_t = double(DFMAEvent[dssd_ba_subev].LEDts) - double(sib_ts[i]);
                   - for (i=0; i<n_sib; i++)  atot=sib_e[i]+d_fr_emax;
           - 处理(叠加信号)：
               - if ((s_d_fr!=0)&&(s_d_ba!=0))
                    - if ((d_fr_PU > 1)&&(d_ba_PU > 1)&&(fd_ba_ts-fd_fr_ts<5)&&(fd_ba_ts-fd_fr_ts>-5))
                        - rec_PU=1;
                        - fdecEv=2;
                        - s_fd_fr=s_d_fr;
                        - s_fd_ba=s_d_ba;
   - 3.2 Recoil correlation(该事件为反冲核时对关联链的处理)
        - if((s_r_fr != 0) && (s_r_ba != 0))(此dssd响应为反冲核)
            - switch (dssd_corr[s_r_fr][s_r_ba].status)(判断反冲核所在像素点的状态)
                - case 0(no recoils so far)
                    - recoil.pu = DFMAEvent[r_fr_subev].pu;
                    - recoil.en = r_fr_emax;
                    - recoil.enb = dssd_fr_subevr_ba_emax;
                    - recoil.ts = r_fr_ts;
                    - recoil.x = crat;
                    - recoil.ptofl= tdssdmcp_fr;
                    - recoil.ptofr= tdssdmcp_fr_r;
                    - recoil.pdetof= tdssdppacde_fr;
                    - recoil.pde=ppacde;
                    - 符合的ｇａｍｍａ（同一事件？？）if(DGSEvent[i].tpe == GE &&  DGSEvent[i].flag == 0 && jj < MAXNUMGE)
                        - recoil.geehi[jj] = DGSEvent[i].ehi;
                        - recoil.tge[jj] = DGSEvent[i].event_timestamp;
                        - recoil.getid[jj] = DGSEvent[i].tid;
                        - recoil.nge = jj;(Ｇｅ计数)
                    - 关联信息
                        - dssd_corr[s_r_fr][s_r_ba].status = 1;
                        - dssd_corr[s_r_fr][s_r_ba].chain.recoil = recoil;
                        - dssd_corr[s_r_fr][s_r_ba].chain.s_fr = s_r_fr;
                        - dssd_corr[s_r_fr][s_r_ba].chain.s_ba=s_r_ba;
                - case 1(recoil last)
                    - 更新该像素点的注入核的信息（具体方法和case 1　相同）
                - case 2()
                    - chain=dssd_corr[s_r_fr][s_r_ba].chain;
                    - chain.corr_type = 1;
                    - tree->Fill();(一条关联链完成)
                    - 更新该像素点的注入核的信息（具体方法和case 1　相同）
                    - 将该像素点的所有衰变信息初始化为0
                    - dssd_corr[s_r_fr][s_r_ba].chain.ndec=0;
                    - dssd_corr[s_r_fr][s_r_ba].status=1;
                    - dssd_corr[s_r_fr][s_r_ba].chain.s_fr = s_r_fr;
                    - dssd_corr[s_r_fr][s_r_ba].chain.s_ba=s_r_ba;
                - default(there is chain already,reset the pixel)
                    - 具体方法和case 2　相同
   - 3.3 Decay correlations(该事件为衰变事件时对关联链的处理)
        - if((s_d_fr != 0) && (s_d_ba != 0) ) decEv=1
        - fdecEv ???
        - while ((decEv==1)||(fdecEv!=0))
            - if (decEv==1)(上述方法判断该事件为衰变事件)
                - DSSD
                    - decay.en = d_fr_emax;
                    - decay.enb = d_ba_emax;
                    - decay.ts = d_fr_ts;
                    - decay.pu_fr=d_fr_PU;(????)
                    - decay.pu_ba=d_ba_PU;(????)
                - clover(同一事件)
                    - decay.nge = XAng;
                    - decay.geehi[j] = clover.ehi[j];
                    - decay.tge[j] = clover.tge[j];
                    - decay.getid[j] = clover.tid[j];
                - SiBox
                    - decay.nsi = n_sib;
                    - decay.nsi=MAXNUMSI;(记最大值)
                    - decay.siehi[j] = sib_e[j];
                    - decay.tsi[j] = sib_ts[j];
                    - decay.sitid[j] = sib_tid[j];
                - decEv=0;
            - else(上述方法判断该事件非衰变事件)
                - if(fdecEv!=0 )
                    - s_d_fr=s_fd_fr;
                    - s_d_ba=s_fd_ba;
                    - decay.en = fd_fr_emax　＝　0;
                    - decay.enb = fd_ba_emax　＝　0;
                    - decay.ts = fd_fr_ts;(****)
                    - if ((fdecEv==1) || (fdecEv == 3))
                        - if(fdecEv == 1)(注入和一代$\alpha$衰变叠加)
                            - pu、t、trace、clover、SIB
                            - fdecEv = 0;
                        - else if(fdecEv == 3)(程序中不存在此情况)
                            - pu、t、trace、clover、SIB
                            - fd_fr_emax = fd_fr_emax2 = 0;
                            - fd_fr_ts = fd_fr_ts2 = 0;
                            - fdecEv = 1;
                    - if (fdecEv==2)（$\alpha$信号叠加）
                        - pu、t、trace、clover、SIB
                        - fdecEv = 0;
            - switch(dssd_corr[s_d_fr][s_d_ba].status)
                - case 0(No recoil before - do nothing):
                    - 直接break
                - case 1(First decay generation):
                    - if (decay.en>0)
                        - decay.time = decay.ts - dssd_corr[s_d_fr][s_d_ba].chain.recoil.ts;
                        - dssd_corr[s_d_fr][s_d_ba].chain.decay[0] = decay;
                        - dssd_corr[s_d_fr][s_d_ba].status++;(接下来是第二代)
                        - dssd_corr[s_d_fr][s_d_ba].chain.ndec=1;
                        - if(decay.time>0)
                            - logtime=log10(10.0*decay.time);
                            - dece_tot=decay.en+sib_emax;
                            - 寿命和衰变能量开门
                                -（if ((logtime>6.5)&&(logtime<10.0)&&(decay.en>6750)&&(decay.en<6900))）
                                - 分析具体的核（Fill Histogram)
                            - electron gamma  (if (logtime > 4 && logtime < 10))
                                - dTgdssd = double(clover.tge[j] - decay.ts);
                                - if(dTgdssd > 100 && dTgdssd <400)  Fill(decay.en,clover.ehi[j])
                - case 2(Second decay generation):
                   - decay.time = decay.ts - dssd_corr[s_d_fr][s_d_ba].chain.decay[0].ts;
                   - decay0_time = dssd_corr[s_d_fr][s_d_ba].chain.decay[0].ts - dssd_corr[s_d_fr][s_d_ba].chain.recoil.ts;
                   - dssd_corr[s_d_fr][s_d_ba].chain.decay[1] = decay;
                   - dssd_corr[s_d_fr][s_d_ba].status++;
                   - dssd_corr[s_d_fr][s_d_ba].chain.ndec=2;
                   - if (decay.time>0)
                        - logtime=log10(10.0*decay.time);
                - case 3
                    - 处理方法和case 2一样
                - case 4
                    - 处理方法和case 2一样    
                - case 5
                    - 处理方法和case 2一样
                - case 6
                    - 处理方法和case 2一样
                - default:
                   - dssd_corr[s_d_fr][s_d_ba].status++;
                   - dssd_corr[s_d_fr][s_d_ba].chain.ndec=7;
 </font>

In [3]:
!jupyter nbconvert Program.ipynb  --to html

[NbConvertApp] Converting notebook Program.ipynb to html

[NbConvertApp] Writing 295965 bytes to Program.html

